<a href="https://colab.research.google.com/github/kdkim2000/JAkorea/blob/main/8__pytorch________gan_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GAN(Generative Adversarial Networks)을 이용한 Fake MNIST 생성

# 라이브러리 import

In [ ]:
import os
import torch
import torchvision
import torch.nn as nn
from torchvision import transforms
from torchvision.utils import save_image

# GPU 사용 체크

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 설정값 지정

In [ ]:
# 설정값 지정
latent_size = 64
hidden_size = 256
image_size = 784
num_epochs = 200
batch_size = 100
sample_dir = 'samples'

# 이미지를 저장할 폴더 생성

In [ ]:
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)

In [ ]:
# Image processing
# transform = transforms.Compose([
#                 transforms.ToTensor(),
#                 transforms.Normalize(mean=(0.5, 0.5, 0.5),   # 3 for RGB channels
#                                      std=(0.5, 0.5, 0.5))])
transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.5],   # 1 for greyscale channels
                                     std=[0.5])])

# MNIST 데이터셋 불러오기

In [ ]:
# MNIST 데이터셋 불러오기
mnist = torchvision.datasets.MNIST(root='../../data/',
                                   train=True,
                                   transform=transform,
                                   download=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../../data/MNIST/raw/train-images-idx3-ubyte.gz to ../../data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../../data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../../data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../../data/MNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [ ]:
data_loader = torch.utils.data.DataLoader(dataset=mnist,
                                          batch_size=batch_size,
                                          shuffle=True)

# 모델 정의

In [ ]:
# Discriminator
D = nn.Sequential(
    nn.Linear(image_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, 1),
    nn.Sigmoid())

# Generator
G = nn.Sequential(
    nn.Linear(latent_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, image_size),
    nn.Tanh())

In [ ]:
D = D.to(device)
G = G.to(device)

# 손실 함수와 옵티마이저 정의

In [ ]:
# 손실함수와 옵티마이저를 정의합니다.
criterion = nn.BCELoss()
d_optimizer = torch.optim.Adam(D.parameters(), lr=0.0002)
g_optimizer = torch.optim.Adam(G.parameters(), lr=0.0002)

In [ ]:
def denorm(x):
    out = (x + 1) / 2
    return out.clamp(0, 1)

def reset_grad():
    d_optimizer.zero_grad()
    g_optimizer.zero_grad()

In [ ]:
# 트레이닝 시작
total_step = len(data_loader)
for epoch in range(num_epochs):
    for i, (images, _) in enumerate(data_loader):
        # image reshape
        images = images.reshape(batch_size, -1).to(device)

        # 정답 label 생성
        real_labels = torch.ones(batch_size, 1).to(device)
        fake_labels = torch.zeros(batch_size, 1).to(device)

        # ---------------------
        #   discriminator 학습
        # ---------------------

        # Compute BCE_Loss using real images where BCE_Loss(x, y): - y * log(D(x)) - (1-y) * log(1 - D(x))
        # Second term of the loss is always zero since real_labels == 1
        outputs = D(images)
        d_loss_real = criterion(outputs, real_labels)
        real_score = outputs

        # Compute BCELoss using fake images
        # First term of the loss is always zero since fake_labels == 0
        z = torch.randn(batch_size, latent_size).to(device)
        fake_images = G(z)
        outputs = D(fake_images)
        d_loss_fake = criterion(outputs, fake_labels)
        fake_score = outputs

        # Backprop and optimize
        d_loss = d_loss_real + d_loss_fake
        reset_grad()
        d_loss.backward()
        d_optimizer.step()

        # ---------------------
        #    generator 학습
        # ---------------------

        # Compute loss with fake images
        z = torch.randn(batch_size, latent_size).to(device)
        fake_images = G(z)
        outputs = D(fake_images)

        # We train G to maximize log(D(G(z)) instead of minimizing log(1-D(G(z)))
        # For the reason, see the last paragraph of section 3. https://arxiv.org/pdf/1406.2661.pdf
        g_loss = criterion(outputs, real_labels)

        # Backprop and optimize
        reset_grad()
        g_loss.backward()
        g_optimizer.step()

        # 학습 과정에 대한 로그 출력
        if (i+1) % 200 == 0:
            print(f'Epoch [{epoch}/{num_epochs}], Step [{i+1}/{total_step}], d_loss: {d_loss.item():.4f}, g_loss: {g_loss.item():.4f}, D(x): {real_score.mean().item():.2f}, D(G(z)): {fake_score.mean().item():.2f}')

    # 리얼 이미지 저장
    if (epoch+1) == 1:
        images = images.reshape(images.size(0), 1, 28, 28)
        save_image(denorm(images), os.path.join(sample_dir, 'real_images.png'))

    # generator가 생성한 fake 이미지 저장
    fake_images = fake_images.reshape(fake_images.size(0), 1, 28, 28)
    save_image(denorm(fake_images), os.path.join(sample_dir, 'fake_images-{}.png'.format(epoch+1)))

Epoch [0/200], Step [200/600], d_loss: 0.1092, g_loss: 5.3722, D(x): 0.93, D(G(z)): 0.02
Epoch [0/200], Step [400/600], d_loss: 0.0297, g_loss: 4.9417, D(x): 0.99, D(G(z)): 0.02
Epoch [0/200], Step [600/600], d_loss: 0.0855, g_loss: 5.5556, D(x): 0.97, D(G(z)): 0.05
Epoch [1/200], Step [200/600], d_loss: 0.2880, g_loss: 3.6351, D(x): 0.91, D(G(z)): 0.08
Epoch [1/200], Step [400/600], d_loss: 0.4455, g_loss: 5.2621, D(x): 0.82, D(G(z)): 0.05
Epoch [1/200], Step [600/600], d_loss: 0.4718, g_loss: 4.1989, D(x): 0.87, D(G(z)): 0.19
Epoch [2/200], Step [200/600], d_loss: 0.2181, g_loss: 3.7314, D(x): 0.93, D(G(z)): 0.07
Epoch [2/200], Step [400/600], d_loss: 0.3555, g_loss: 2.9461, D(x): 0.90, D(G(z)): 0.15
Epoch [2/200], Step [600/600], d_loss: 0.5186, g_loss: 2.1262, D(x): 0.80, D(G(z)): 0.14
Epoch [3/200], Step [200/600], d_loss: 0.5719, g_loss: 2.6438, D(x): 0.77, D(G(z)): 0.11
Epoch [3/200], Step [400/600], d_loss: 0.2926, g_loss: 3.3188, D(x): 0.86, D(G(z)): 0.06
Epoch [3/200], Step [